In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

# Model

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)
    def forward(self, x):
        return self.linear(x)

In [6]:
model = SoftmaxClassifierModel()
model

SoftmaxClassifierModel(
  (linear): Linear(in_features=3, out_features=3, bias=True)
)

In [7]:
# optimizer 지정
optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer 

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [9]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.cross_entropy(prediction, y_train)

        # cost로 H(x)개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epcoh {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [11]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    # max : 최대값 및 인덱스 반환(인덱스만 추출[1]_레이블)
    predicted_classes = prediction.max(1)[1]
    # 같은 레이블 갯수 카운트
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    # 맞은 갯수
    print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) * 100, cost.item()
    ))

In [18]:
test_pred = model(x_train)
test_pred

tensor([[-0.2109, -0.1397,  0.2235],
        [-0.1823, -0.1716,  0.0367],
        [ 0.1153,  0.5762, -0.3795],
        [ 0.0238,  0.1385, -0.5450],
        [-0.2166, -0.6731, -0.5023],
        [ 0.3843,  1.3560, -0.6090],
        [ 0.0524,  0.1066, -0.7318],
        [ 0.0810,  0.0748, -0.9186]], grad_fn=<AddmmBackward0>)

In [23]:
test_classes = test_pred.max(1)
print(test_classes)

torch.return_types.max(
values=tensor([ 0.2235,  0.0367,  0.5762,  0.1385, -0.2166,  1.3560,  0.1066,  0.0810],
       grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 1, 1, 0, 1, 1, 0]))


In [17]:
test_pred.shape

torch.Size([3])

In [10]:
# MLE(최대우도추정)을 통해 최적의 우도가 찾아지는 과정
train(model, optimizer, x_train, y_train)

Epcoh    0/20 Cost: 2.203667
Epcoh    1/20 Cost: 1.199645
Epcoh    2/20 Cost: 1.142985
Epcoh    3/20 Cost: 1.117769
Epcoh    4/20 Cost: 1.100901
Epcoh    5/20 Cost: 1.089523
Epcoh    6/20 Cost: 1.079872
Epcoh    7/20 Cost: 1.071320
Epcoh    8/20 Cost: 1.063325
Epcoh    9/20 Cost: 1.055720
Epcoh   10/20 Cost: 1.048378
Epcoh   11/20 Cost: 1.041245
Epcoh   12/20 Cost: 1.034285
Epcoh   13/20 Cost: 1.027478
Epcoh   14/20 Cost: 1.020813
Epcoh   15/20 Cost: 1.014279
Epcoh   16/20 Cost: 1.007872
Epcoh   17/20 Cost: 1.001586
Epcoh   18/20 Cost: 0.995419
Epcoh   19/20 Cost: 0.989365


In [12]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


## Learning Rate

Gradient Descent 에서의 $\alpha$ 값

`optimizer = optim.SGD(model.parameters(), lr=0.1)` 에서 `lr=0.1` 이다

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [24]:
model = SoftmaxClassifierModel()
model

SoftmaxClassifierModel(
  (linear): Linear(in_features=3, out_features=3, bias=True)
)

In [25]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [26]:
train(model, optimizer, x_train, y_train)

Epcoh    0/20 Cost: 1.280268
Epcoh    1/20 Cost: 976950.687500
Epcoh    2/20 Cost: 1279135.125000
Epcoh    3/20 Cost: 1198379.125000
Epcoh    4/20 Cost: 1098825.625000
Epcoh    5/20 Cost: 1968197.625000
Epcoh    6/20 Cost: 284763.156250
Epcoh    7/20 Cost: 1532260.125000
Epcoh    8/20 Cost: 1651504.000000
Epcoh    9/20 Cost: 521878.437500
Epcoh   10/20 Cost: 1397263.125000
Epcoh   11/20 Cost: 750986.250000
Epcoh   12/20 Cost: 918691.500000
Epcoh   13/20 Cost: 1487888.125000
Epcoh   14/20 Cost: 1582260.125000
Epcoh   15/20 Cost: 685818.062500
Epcoh   16/20 Cost: 1140048.750000
Epcoh   17/20 Cost: 940566.562500
Epcoh   18/20 Cost: 931638.125000
Epcoh   19/20 Cost: 1971322.625000


## Data Preprocessing (데이터 전처리)

데이터를 zero-center하고 normalize하자.

In [27]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [28]:
# x_train의 평균
mu = x_train.mean(dim=0)
mu

tensor([84.8000, 84.6000, 85.6000])

In [29]:
# x_train의 표준편차
sigma = x_train.std(dim=0)
sigma

tensor([11.0544, 12.2393, 12.6214])

In [30]:
norm_x_train = (x_train - mu) / sigma
norm_x_train

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])

In [31]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [32]:
model = MultivariateLinearRegressionModel()

In [33]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [34]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [35]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29547.708984
Epoch    1/20 Cost: 18766.755859
Epoch    2/20 Cost: 11969.090820
Epoch    3/20 Cost: 7648.307129
Epoch    4/20 Cost: 4891.651367
Epoch    5/20 Cost: 3129.900879
Epoch    6/20 Cost: 2003.104126
Epoch    7/20 Cost: 1282.158936
Epoch    8/20 Cost: 820.807800
Epoch    9/20 Cost: 525.553101
Epoch   10/20 Cost: 336.587830
Epoch   11/20 Cost: 215.644012
Epoch   12/20 Cost: 138.233063
Epoch   13/20 Cost: 88.683517
Epoch   14/20 Cost: 56.965141
Epoch   15/20 Cost: 36.658813
Epoch   16/20 Cost: 23.656675
Epoch   17/20 Cost: 15.329287
Epoch   18/20 Cost: 9.994032
Epoch   19/20 Cost: 6.574042
